# Building a Role-Based RAG Pipeline with Redis

This notebook demonstrates a simplified setup for a **Role-Based Retrieval Augmented Generation (RAG)** pipeline, where:

1. Each **User** has one or more **roles**.
2. Knowledge base **Documents** in Redis are tagged with the official roles that can access them (`allowed_roles`).
3. A unified **query flow** ensures a user only sees documents that match at least one of their roles.

![Role Based RAG](https://raw.githubusercontent.com/redis-developer/redis-ai-resources/main/assets/role-based-rag.png)


## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/RAG/07_user_role_based_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q "redisvl>=0.4.1" openai langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.6 MB/s eta 0:00:00


## 1. High-Level Data Flow & Setup

1. **User Creation & Role Management**
   - A user is stored at `user:{user_id}` in Redis with a JSON structure containing the user’s roles.
   - We can create, update, or delete users as needed.
   - **This serves as a simple look up layer and should NOT replace your production-ready auth API flow**

2. **Document Storage**
   - Documents chunks are stored at `doc:{doc_id}:{chunk_id}` in Redis as JSON.
   - Each document chunk includes fields such as `doc_id`, `chunk_id`, `content`, `allowed_roles`, and an `embedding` (for vector similarity).

3. **Querying / Search**
   - User roles are retrieved from Redis.
   - We perform a vector similarity search (or any other type of retrieval) on the documents.
   - We filter the results so that only documents whose `allowed_roles` intersect with the user’s roles are returned.

4. **RAG Integration**
   - The returned documents can be fed into a Large Language Model (LLM) to provide context and generate an answer.

First, we’ll set up our Python environment and Redis connection.


### Download Documents
Running remotely or in collab? Run this cell to download the necessary datasets.

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mkdir -p resources
!mv temp_repo/python-recipes/RAG/resources/aapl-10k-2023.pdf resources/
!mv temp_repo/python-recipes/RAG/resources/2022-chevy-colorado-ebrochure.pdf resources/
!rm -rf temp_repo

### Run Redis Stack

For this tutorial you will need a running instance of Redis if you don't already have one.

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [4]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

In [5]:
import os

from redis import Redis

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

# Connect to Redis (adjust host/port if needed)
redis_client = Redis.from_url(REDIS_URL)
redis_client.ping()

print("Successfully connected to Redis")

Successfully connected to Redis


## 2. User Management

Below is a simple `User` class that stores a user in Redis as JSON. We:

- Use a Redis key of the form `user:{user_id}`.
- Store fields like `user_id`, `roles`, etc.
- Provide CRUD methods (Create, Read, Update, Delete) for user objects.

**Data Structure Example**
```json
{
  "user_id": "alice",
  "roles": ["finance", "manager"]
}
```

We'll also include some basic checks to ensure we don't add duplicate roles, handle empty role lists, etc.


In [6]:
from typing import List, Optional
from enum import Enum


class UserRoles(str, Enum):
    FINANCE = "finance"
    MANAGER = "manager"
    EXECUTIVE = "executive"
    HR = "hr"
    SALES = "sales"
    PRODUCT = "product"


class User:
    """
    User class for storing user data in Redis.

    Each user has:
    - user_id (string)
    - roles (list of UserRoles)

    Key in Redis: user:{user_id}
    """
    def __init__(
        self,
        redis_client: Redis,
        user_id: str,
        roles: Optional[List[UserRoles]] = None
    ):
        self.redis_client = redis_client
        self.user_id = user_id
        self.roles = roles or []

    @property
    def key(self) -> str:
        return f"user:{self.user_id}"

    def exists(self) -> bool:
        """Check if the user key exists in Redis."""
        return self.redis_client.exists(self.key) == 1

    def create(self):
        """
        Create a new user in Redis. Fails if user already exists.
        """
        if self.exists():
            raise ValueError(f"User {self.user_id} already exists.")

        self.save()

    def save(self):
        """
        Save (create or update) the user data in Redis.
        If user does not exist, it will be created.
        """
        data = {
            "user_id": self.user_id,
            "roles": [UserRoles(role).value for role in set(self.roles)] # ensure roles are unique and convert to strings
        }
        self.redis_client.json().set(self.key, ".", data)

    @classmethod
    def get(cls, redis_client: Redis, user_id):
        """
        Retrieve a user from Redis.
        """
        key = f"user:{user_id}"
        data = redis_client.json().get(key)
        if not data:
            return None
        # Convert string roles back to UserRoles enum
        roles = [UserRoles(role) for role in data.get("roles", [])]
        return cls(redis_client, data["user_id"], roles)

    def update_roles(self, roles: List[UserRoles]):
        """
        Overwrite the user's roles in Redis.
        """
        self.roles = roles
        self.save()

    def add_role(self, role: UserRoles):
        """Add a single role to the user."""
        if role not in self.roles:
            self.roles.append(role)
            self.save()

    def remove_role(self, role: UserRoles):
        """Remove a single role from the user."""
        if role in self.roles:
            self.roles.remove(role)
            self.save()

    def delete(self):
        """Delete this user from Redis."""
        self.redis_client.delete(self.key)

    def __repr__(self):
        return f"<User user_id={self.user_id}, roles={[UserRoles(role).value for role in self.roles]}>"


### Example usage of User class

In [7]:
# Example usage of the User class

# Let's create a new user
alice = User(redis_client, "alice", roles=["finance", "manager"])

# We'll save the user in Redis
try:
    alice.create()
    print("User 'alice' created.")
except ValueError as e:
    print(e)

# Retrieve the user
alice_obj = User.get(redis_client, "alice")
print("Retrieved:", alice_obj)

# Add another role
alice_obj.add_role("executive")
print("After adding 'executive':", alice_obj)

# Remove a role
alice_obj.remove_role("manager")
print("After removing 'manager':", alice_obj)


User 'alice' created.
Retrieved: <User user_id=alice, roles=['finance', 'manager']>
After adding 'executive': <User user_id=alice, roles=['finance', 'manager', 'executive']>
After removing 'manager': <User user_id=alice, roles=['finance', 'executive']>


In [8]:
# Take a peek at the user object itself
alice

<User user_id=alice, roles=['finance', 'manager']>

In [9]:
# Create one more user
larry = User(redis_client, "larry", roles=["product"])
larry.create()

>💡 Using a cloud DB? Take a peek at your instance using [RedisInsight](https://redis.io/insight) to see what user data is in place.

## 3. Document Management (Using LangChain)

Here, we'll use **LangChain** for document loading, chunking, and vectorizing. Then, we’ll **store documents** in Redis as JSON. Each document will look like:

```json
{
  "doc_id": "123",
  "chunk_id": "123",
  "path": "resources/doc.pdf",
  "title": "Quarterly Finance Report",
  "content": "Some text...",
  "allowed_roles": ["finance", "executive"],
  "embedding": [0.12, 0.98, ...]  
}
```

### Building a document knowledge base
We will create a `KnowledgeBase` class to encapsulate document processing logic and search. The class will handle:
1. Document ingest and chunking
2. Role tagging with a simple str-based rule (likely custom depending on use case)
3. Retrieval over the entire document corpus adhering to provided user roles


In [23]:
from typing import List, Optional, Dict, Any, Set
from pathlib import Path
import uuid

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from redisvl.index import SearchIndex
from redisvl.query import VectorQuery
from redisvl.query.filter import FilterExpression, Tag
from redisvl.utils.vectorize import OpenAITextVectorizer


class KnowledgeBase:
    """Manages document processing, embedding, and storage in Redis."""

    def __init__(
        self,
        redis_client,
        embeddings_model: str = "text-embedding-3-small",
        chunk_size: int = 2500,
        chunk_overlap: int = 100
    ):
        self.redis_client = redis_client
        self.embeddings = OpenAITextVectorizer(model=embeddings_model)
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
        )

        # Initialize document search index
        self.index = self._create_search_index()

    def _create_search_index(self) -> SearchIndex:
        """Create the Redis search index for documents."""
        schema = {
            "index": {
                "name": "docs",
                "prefix": "doc",
                "storage_type": "json"
            },
            "fields": [
                {
                    "name": "doc_id",
                    "type": "tag",
                },
                {
                    "name": "chunk_id",
                    "type": "tag",
                },
                {
                    "name": "allowed_roles",
                    "path": "$.allowed_roles[*]",
                    "type": "tag",
                },
                {
                    "name": "content",
                    "type": "text",
                },
                {
                    "name": "embedding",
                    "type": "vector",
                    "attrs": {
                        "dims": self.embeddings.dims,
                        "distance_metric": "cosine",
                        "algorithm": "flat",
                        "datatype": "float32"
                    }
                }
            ]
        }
        index = SearchIndex.from_dict(schema, redis_client=self.redis_client)
        index.create()
        return index

    def ingest(self, doc_path: str, allowed_roles: Optional[List[str]] = None) -> str:
        """
        Load a document, chunk it, create embeddings, and store in Redis.
        Returns the document ID.
        """
        # Generate document ID
        doc_id = str(uuid.uuid4())
        path = Path(doc_path)

        if not path.exists():
            raise FileNotFoundError(f"Document not found: {doc_path}")

        # Load and chunk document
        loader = PyPDFLoader(str(path))
        pages = loader.load()
        chunks = self.text_splitter.split_documents(pages)
        print(f"Extracted {len(chunks)} for doc {doc_id} from file {str(path)}", flush=True)

        # If roles not provided, determine from filename
        if allowed_roles is None:
            allowed_roles = self._determine_roles(path)

        # Prepare chunks for Redis
        data, keys = [], []
        for i, chunk in enumerate(chunks):
            # Create embedding w/ openai
            embedding = self.embeddings.embed(chunk.page_content)

            # Prepare chunk payload
            chunk_id = f"chunk_{i}"
            key = f"doc:{doc_id}:{chunk_id}"
            data.append({
                "doc_id": doc_id,
                "chunk_id": chunk_id,
                "path": str(path),
                "content": chunk.page_content,
                "allowed_roles": list(allowed_roles),
                "embedding": embedding,
            })
            keys.append(key)

        # Store in Redis
        _ = self.index.load(data=data, keys=keys)
        print(f"Loaded {len(chunks)} chunks for document {doc_id}")
        return doc_id

    def _determine_roles(self, file_path: Path) -> Set[str]:
        """Determine allowed roles based on file path and name patterns."""
        # Customize based on use case and business logic
        ROLE_PATTERNS = {
            ('10k', 'financial', 'earnings', 'revenue'):
                {'finance', 'executive'},
            ('brochure', 'spec', 'product', 'manual'):
                {'product', 'sales'},
            ('hr', 'handbook', 'policy', 'employee'):
                {'hr', 'manager'},
            ('sales', 'pricing', 'customer'):
                {'sales', 'manager'}
        }

        filename = file_path.name.lower()
        roles = {
            role for terms, roles in ROLE_PATTERNS.items()
            for role in roles
            if any(term in filename for term in terms)
        }
        return roles or {'executive'}

    @staticmethod
    def role_filter(user_roles: List[str]) -> FilterExpression:
        """Generate a Redis filter based on provided user roles."""
        return Tag("allowed_roles") == user_roles

    def search(self, query: str, user_roles: List[str], top_k: int = 5) -> List[Dict[str, Any]]:
        """
        Search for documents matching the query and user roles.
        Returns list of matching documents.
        """
        # Create query vector
        query_vector = self.embeddings.embed(query)

        # Build role filter
        roles_filter = self.role_filter(user_roles)

        # Execute search
        return self.index.query(
            VectorQuery(
                vector=query_vector,
                vector_field_name="embedding",
                filter_expression=roles_filter,
                return_fields=["doc_id", "chunk_id", "allowed_roles", "content"],
                num_results=top_k,
                dialect=4
            )
        )


Load a document into the knowledge base.

In [12]:
kb = KnowledgeBase(redis_client)

doc_id = kb.ingest("resources/2022-chevy-colorado-ebrochure.pdf")
print(f"Loaded all chunks for {doc_id}", flush=True)

21:09:47 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Extracted 34 for doc f2c7171a-16cc-4aad-a777-ed7202bd7212 from file resources/2022-chevy-colorado-ebrochure.pdf
21:09:49 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:09:49 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:09:50 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:09:50 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:09:51 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:09:51 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:09:52 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:09:52 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:09:53 httpx INFO   HT

## 4. User Query Flow

Now that we have our User DB and our Vector DB loaded in Redis. We will perform:

1. **Vector Similarity Search** on `embedding`.
2. A metadata **Filter** based on `allowed_roles`.
3. Return top-k matching document chunks.

This is implemented below.


In [13]:
def user_query(user_id: str, query: str):
    """
    Placeholder for a search function.
    1. Load the user's roles.
    2. Perform a vector search for docs.
    3. Filter docs that match at least one of the user's roles.
    4. Return top-K results.
    """
    # 1. Load & validate user roles
    user_obj = User.get(redis_client, user_id)
    if not user_obj:
        raise ValueError(f"User {user_id} not found.")

    roles = set([role.value for role in user_obj.roles])
    if not roles:
        raise ValueError(f"User {user_id} does not have any roles.")

    # 2. Retrieve document chunks
    results = kb.search(query, roles)

    if not results:
      raise ValueError(f"No available documents found for {user_id}")

    return results

### Search examples

Search with a non-existent user.

In [14]:
# NBVAL_SKIP
results = user_query("tyler", query="What is the make and model of the vehicle here?")

ValueError: User tyler not found.

Create user for Tyler.

In [15]:
# NBVAL_SKIP
tyler = User(redis_client, "tyler", roles=["sales", "engineering"])
tyler.create()

ValueError: 'engineering' is not a valid UserRoles

In [16]:
# Try again but this time with valid roles
tyler = User(redis_client, "tyler", roles=["sales"])
tyler.create()

In [17]:
tyler

<User user_id=tyler, roles=['sales']>

In [18]:
# Query with valid user
results = user_query(
    tyler.user_id,
    query="What is the make and model of the vehicle here?"
)
results[:3]

21:10:21 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[{'id': 'doc:f2c7171a-16cc-4aad-a777-ed7202bd7212:chunk_13',
  'vector_distance': '0.60664498806',
  'doc_id': '["f2c7171a-16cc-4aad-a777-ed7202bd7212"]',
  'chunk_id': '["chunk_13"]',
  'allowed_roles': '["sales","product"]'},
 {'id': 'doc:f2c7171a-16cc-4aad-a777-ed7202bd7212:chunk_11',
  'vector_distance': '0.613630235195',
  'doc_id': '["f2c7171a-16cc-4aad-a777-ed7202bd7212"]',
  'chunk_id': '["chunk_11"]',
  'allowed_roles': '["sales","product"]'},
 {'id': 'doc:f2c7171a-16cc-4aad-a777-ed7202bd7212:chunk_19',
  'vector_distance': '0.62441521883',
  'doc_id': '["f2c7171a-16cc-4aad-a777-ed7202bd7212"]',
  'chunk_id': '["chunk_19"]',
  'allowed_roles': '["sales","product"]'}]

Search with a valid user, but incorrect roles.

In [19]:
# NBVAL_SKIP
print(alice, "\n")

# Query with valid user
results = user_query(
    alice.user_id, query="What is the make and model of the vehicle here?"
)
results

<User user_id=alice, roles=['finance', 'manager']> 

21:10:24 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


ValueError: No available documents found for alice

Empty results because there are no documents available for Alice to view. Add some.

In [20]:
# Add a document that Alice will have access to
kb.ingest("resources/aapl-10k-2023.pdf")

Extracted 155 for doc 42b58f50-d689-4a36-8977-e8ca1a183446 from file resources/aapl-10k-2023.pdf
21:10:32 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:32 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:32 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:32 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:32 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:33 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:33 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:33 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:34 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:10:34 httpx INFO   HTTP Request: POS

'42b58f50-d689-4a36-8977-e8ca1a183446'

In [22]:
# Query with valid user
results = user_query(
    alice.user_id,
    query="What was the total revenue amount for Apple according to their 10k?"
)
results[:3]

21:11:30 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[{'id': 'doc:42b58f50-d689-4a36-8977-e8ca1a183446:chunk_81',
  'vector_distance': '0.343286693096',
  'doc_id': '["42b58f50-d689-4a36-8977-e8ca1a183446"]',
  'chunk_id': '["chunk_81"]',
  'allowed_roles': '["finance","executive"]'},
 {'id': 'doc:42b58f50-d689-4a36-8977-e8ca1a183446:chunk_68',
  'vector_distance': '0.353579521179',
  'doc_id': '["42b58f50-d689-4a36-8977-e8ca1a183446"]',
  'chunk_id': '["chunk_68"]',
  'allowed_roles': '["finance","executive"]'},
 {'id': 'doc:42b58f50-d689-4a36-8977-e8ca1a183446:chunk_72',
  'vector_distance': '0.354550600052',
  'doc_id': '["42b58f50-d689-4a36-8977-e8ca1a183446"]',
  'chunk_id': '["chunk_72"]',
  'allowed_roles': '["finance","executive"]'}]

## 5. Implementing Role-Based RAG from scratch
*with OpenAI and Redis*

In [39]:
from openai import OpenAI
from typing import List, Optional
import os

from redisvl.extensions.session_manager import StandardSessionManager


class RAGChatManager:
    """
    Manages RAG-enhanced chat interactions with role-based access control and chat history.

    Attributes:
        kb: A KnowledgeBase instance for searching documents
        client: An OpenAI client for chat completions
        model: Name of OpenAI model to use
        sessions: Dict to store active chat sessions
        system_prompt: The default system prompt
    """

    def __init__(
        self,
        knowledge_base: "KnowledgeBase",
        openai_api_key: Optional[str] = None,
        openai_model: str = "gpt-4",
        system_prompt: str = "You are a helpful chatbot assistant with access to knowledge base documents"
    ):
        """Initialize the RAG chat manager."""
        self.kb = knowledge_base
        self.client = OpenAI(api_key=openai_api_key or os.getenv("OPENAI_API_KEY"))
        self.model = openai_model
        self.sessions = {}
        self.system_prompt = system_prompt

    def user_roles(self, user_id: str) -> set:
        """
        Get and validate user roles.

        Args:
            user_id: User identifier

        Returns:
            Set of user roles

        Raises:
            ValueError: If user not found or has no roles
        """
        user_obj = User.get(self.kb.redis_client, user_id)
        if not user_obj:
            raise ValueError(f"User {user_id} not found.")

        roles = set([role.value for role in user_obj.roles])
        if not roles:
            raise ValueError(f"User {user_id} does not have any roles.")

        return roles

    def start_session(self, user_id: str) -> None:
        """
        Start a new chat session for a user.

        Args:
            user_id: User identifier
        """
        if user_id not in self.sessions:
            self.sessions[user_id] = StandardSessionManager(
                name=f"session:{user_id}",
                redis_client=self.kb.redis_client
              )

    def prep_msgs(
        self,
        user_id: str,
        system_prompt: str,
        context: str,
        query: str
    ) -> List[dict]:
        """
        Get chat history messages including system prompt.

        Args:
            user_id: User identifier for the session
            system_prompt: Optional system prompt to prepend
            context: Relevant context fetched from the knowledge base
            query: Original user question

        Returns:
            List of message dictionaries
        """
        messages = [{"role": "system", "content": system_prompt}]

        if user_id in self.sessions:
            messages.extend(self.sessions[user_id].get_recent())

        messages.append({
            "role": "user",
            "content": f"""Context information is below.
            ---------------------
            {context}
            ---------------------
            Given the context information above and the chat conversation history, please answer the question faithfully: {query}"""
        })

        for msg in messages:
            if msg["role"] == "llm":
                msg["role"] = "assistant"

        return messages

    def chat(self, user_id: str, system_prompt: Optional[str] = None) -> None:
        """
        Start an interactive chat loop with the user.

        Args:
            user_id: User identifier
            system_prompt: Optional system prompt

        The loop continues until user types 'exit' or 'quit'
        """
        self.start_session(user_id)

        print("Starting chat session with GPT4. Type 'exit' or 'quit' to end the session.")
        while True:
            query = input("\nYou: ").strip()

            if query.lower() in ['exit', 'quit']:
                print("\nEnding chat session...")
                break

            response = self.answer(query, user_id, system_prompt)
            print(f"\nAssistant: {response}")

    def answer(
        self,
        query: str,
        user_id: str,
        system_prompt: Optional[str] = None
    ) -> str:
        """
        Process a chat message with RAG enhancement and role-based access.

        If any exception occurs at any stage (roles, document search, LLM call),
        we do NOT store anything in the session and simply return the error.
        Otherwise, we store the query and the response (including 'no docs found' case).

        Args:
            query: User's question
            user_id: User identifier
            system_prompt: Optional system prompt

        Returns:
            AI response string or error message
        """

        # Start or retrieve an existing session for user
        self.start_session(user_id)

        try:
            # 1. Validate user roles
            roles = self.user_roles(user_id)

            # 2. Use provided system prompt or default
            system_prompt = system_prompt or self.system_prompt

            # 3. Search for relevant documents
            docs = self.kb.search(query, roles)

            # 4. If no documents, store & return early
            if not docs:
                no_docs_msg = (
                    "I couldn't find any relevant documents you have permission to access. "
                    "Please try rephrasing your question or contact an administrator if you believe this is an error."
                )
                self.sessions[user_id].store(query, no_docs_msg)
                return no_docs_msg

            # 5. Prepare context and messages for the LLM
            context = "\n\n".join([doc.get("content", "") for doc in docs])
            messages = self.prep_msgs(
                user_id=user_id,
                system_prompt=system_prompt,
                context=context,
                query=query
            )

            # 6. Generate response from the model
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            ai_response = response.choices[0].message.content

            # 7. Store query and LLM response
            self.sessions[user_id].store(query, ai_response)

            return ai_response

        except Exception as e:
            # Catch any exception; do not store anything, just return the error.
            return f"I encountered an error: {str(e)}"


### Session-aware, role-based RAG

In [40]:
bot = RAGChatManager(kb)

In [41]:
bot.answer("What is the make and model of the vehicle?", user_id="alice")

21:20:45 redisvl.index.index INFO   Index already exists, not overwriting.
21:20:45 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:20:47 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


"The context information provided does not contain any details about a vehicle's make and model."

In [42]:
bot.answer("What is the make and model of the vehicle?", user_id="tyler")

21:20:50 redisvl.index.index INFO   Index already exists, not overwriting.
21:20:50 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:20:51 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'The make and model of the vehicle is Chevrolet Colorado.'

In [43]:
bot.answer("What year is it?", user_id="tyler")

21:20:54 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:20:55 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'The vehicle is from the year 2022.'

In [44]:
# NBVAL_SKIP
bot.chat(user_id="tyler")

Starting chat session with GPT4. Type 'exit' or 'quit' to end the session.

You: What is the towing capacity of the truck?
21:22:10 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:22:14 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

Assistant: The towing capacity of the truck varies depending on the specific model and engine. The 2.5L DOHC I-4 engine has a maximum towing weight rating of 3,500 lbs, the 3.6L DOHC V6 engine can tow up to 7,000 lbs, and the Duramax 2.8L Turbo-Diesel I-4 engine has a maximum towing weight rating of 7,700 lbs. You should always check the specific towing capacity of your vehicle and never exceed it, as this can lead to vehicle damage or unsafe driving conditions.

You: Is it generally safe to drive? What safety features are available?
21:22:28 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
21:22:39 httpx INFO   HTTP Request: POST h

## 6. Summary & Next Steps

In this notebook, we set up a **basic** for a Role-Based RAG system:

1. **Users** (with `roles`) stored in Redis via JSON.
2. **Documents** (with `allowed_roles`) loaded, parsed, embedded and also stored in Redis.
3. A user search pipeline that honors user roles when retrieving documents.


This approach ensures that **only documents** whose roles match the user’s roles are returned.


With these building blocks in place, you can integrate an LLM to supply a context from the returned docs, producing a robust retrieval-augmented generation pipeline with role-based access controls.
